In [70]:
import pandas as pd

In [71]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

# select the dictionary for the Celtics
celtics = [team for team in nba_teams if team['abbreviation']=='BOS'][0]
celtics_id = celtics['id']
print(f'celtics_id: {celtics_id}')

celtics_id: 1610612738


In [72]:
# query for the last regular season game where BOS palyed
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

game_finder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id,
                                               season_nullable=Season.default,
                                               season_type_nullable=SeasonType.regular)

games_dict = game_finder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

print(f'Searching throguh {len(games)} games(s) for the game_id of {game_id} where {game_matchup}')

Searching throguh 82 games(s) for the game_id of 0022101223 where BOS @ MEM


## Retrieving the play-by-play data

In [73]:
# query for the play-by-play of the msot recent regular seaons game
from nba_api.stats.endpoints import playbyplay
df = playbyplay.PlayByPlay(game_id).get_data_frames()[0]
df.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0022101223,2,12,0,1,7:18 PM,12:00,None,Start of 1st Period (7:18 PM EST),None,None,None
1,0022101223,4,10,0,1,7:18 PM,12:00,Jump Ball Tillman vs. Horford: Tip to Brown,None,None,None,None
2,0022101223,7,5,1,1,7:18 PM,11:44,Tillman STEAL (1 STL),None,Horford Bad Pass Turnover (P1.T1),None,None
3,0022101223,9,2,79,1,7:19 PM,11:31,MISS Melton 29' 3PT Pullup Jump Shot,None,None,None,None
4,0022101223,10,4,0,1,7:19 PM,11:28,None,None,Theis REBOUND (Off:0 Def:1),None,None


In [74]:
# change the option of the pandas df
import pandas as pd
pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_rows', 250)

In [75]:
# list unqiue values in the df['EVENTMSGTYPE']
print(f'EVENTMSGTYPE: {sorted(df.EVENTMSGTYPE.unique())}')

EVENTMSGTYPE: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 18]


In [76]:
# Enum
from enum import Enum

class EventMsgType(Enum):
    FIELD_GOAL_MADE = 1
    FIELD_GOAL_MISSED = 2
    FREE_THROW_attempt =3
    REBOUND = 4
    TURNVOER = 5
    FOUL =6
    VIOLATION = 7
    SUBSTITUTION = 8
    TIMEOUT = 9
    JUMP_BALL = 10
    EJECTION = 11
    PERIOD_BEGIN = 12
    PERIOD_END = 13

In [77]:
# pull the data for a specific EVENTMSGTYPE
df.loc[df['EVENTMSGTYPE'] == 1].head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
6,0022101223,13,1,1,1,7:19 PM,11:20,None,None,Brown 26' 3PT Jump Shot (3 PTS) (Theis 1 AST),3 - 0,-3
11,0022101223,20,1,5,1,7:20 PM,10:36,None,None,Horford 2' Layup (2 PTS),5 - 0,-5
19,0022101223,30,1,52,1,7:21 PM,9:51,None,None,Tatum 1' Alley Oop Dunk (2 PTS) (Theis 2 AST),7 - 1,-6
22,0022101223,34,1,1,1,7:22 PM,9:28,Melton 3PT Jump Shot (3 PTS) (Anderson 1 AST),None,None,7 - 4,-3
26,0022101223,40,1,80,1,7:22 PM,9:08,None,None,Brown 22' Step Back Jump Shot (5 PTS),9 - 4,-5


## get all play by play

In [78]:
game_finder_BOS = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)

games = game_finder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22021,1610612738,BOS,Boston Celtics,0022101223,2022-04-10,BOS @ MEM,W,240,139,...,1.000,14,42,56,34,5,2,15,20,29.0
1,22021,1610612738,BOS,Boston Celtics,0022101198,2022-04-07,BOS @ MIL,L,240,121,...,0.737,8,36,44,30,8,4,16,15,-6.0
2,22021,1610612738,BOS,Boston Celtics,0022101193,2022-04-06,BOS @ CHI,W,241,117,...,0.636,8,34,42,29,11,6,11,18,23.0
3,22021,1610612738,BOS,Boston Celtics,0022101166,2022-04-03,BOS vs. WAS,W,240,144,...,0.692,7,32,39,39,6,5,7,20,42.0
4,22021,1610612738,BOS,Boston Celtics,0022101153,2022-04-01,BOS vs. IND,W,239,128,...,0.929,11,29,40,29,7,4,15,19,5.0


In [86]:
game_ids = games['GAME_ID'].unique()
game_ids = game_ids[:2]
game_ids

array(['0022101223', '0022101198'], dtype=object)

In [87]:
# create dictionary
game_id_dict = {game_id : pd.DataFrame() for game_id in game_ids}
game_id_dict

{'0022101223': Empty DataFrame
 Columns: []
 Index: [],
 '0022101198': Empty DataFrame
 Columns: []
 Index: []}

In [88]:
from nba_api.stats.endpoints import playbyplayv2

raw_data = []
for game_id in game_ids:
    pbp = playbyplayv2.PlayByPlayV2(game_id)
    pbp = pbp.get_data_frames()[0]
raw_data = pd.concat(raw_data)

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [90]:
from nba_py.league import GameLog
stats = GameLog(season='2020-21', season_type='Regular Season', player_or_team='P').overall()

ModuleNotFoundError: No module named 'nba_py'

In [93]:
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats
df = get_roster('GSW', 2019)
print(df)

    NUMBER             PLAYER POS HEIGHT  WEIGHT BIRTH_DATE NATIONALITY  \
0        2        Jordan Bell   C    6-8     216 1995-01-07          US   
1       12       Andrew Bogut   C    7-0     260 1984-11-28          AU   
2        4         Quinn Cook  PG    6-1     180 1993-03-23          US   
3        0   DeMarcus Cousins   C   6-10     270 1990-08-13          US   
4       30      Stephen Curry  PG    6-2     185 1988-03-14          US   
5       32  Marcus Derrickson  PF    6-7     249 1996-02-01          US   
6       35       Kevin Durant  SF   6-10     240 1988-09-29          US   
7       10        Jacob Evans  SG    6-4     210 1997-06-18          US   
8       23     Draymond Green  PF    6-6     230 1990-03-04          US   
9        9     Andre Iguodala  SF    6-6     215 1984-01-28          US   
10      21      Jonas Jerebko  PF   6-10     231 1987-03-02          SE   
11      15       Damian Jones   C   6-11     245 1995-06-30          US   
12       1         Damion

In [96]:
from basketball_reference_scraper.pbp import get_pbp

s = get_pbp('2020-01-13', 'CHI', 'BOS')
s.head()

,QUARTER,TIME_REMAINING,CHICAGO_ACTION,BOSTON_ACTION,CHICAGO_SCORE,BOSTON_SCORE
0,1,11:35.0,NaN,E. Freedom makes 2-pt layup from 2 ft,0,2
1,1,11:17.0,L. Markkanen misses 3-pt jump shot from 26 ft,NaN,0,2
2,1,11:15.0,NaN,Defensive rebound by E. Freedom,0,2
3,1,11:11.0,NaN,Personal foul by D. Gafford (drawn by J. Brown),0,2
4,1,11:00.0,NaN,J. Tatum makes 2-pt jump shot from 16 ft (assist by G. Hayward),0,4


In [98]:
from basketball_reference_scraper.seasons import get_schedule, get_standings

s = get_schedule(2018, playoffs=False)
s

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
0,2017-10-17,Boston Celtics,99,Cleveland Cavaliers,102
1,2017-10-17,Houston Rockets,122,Golden State Warriors,121
2,2017-10-18,Charlotte Hornets,90,Detroit Pistons,102
3,2017-10-18,Brooklyn Nets,131,Indiana Pacers,140
4,2017-10-18,Miami Heat,109,Orlando Magic,116
5,2017-10-18,Philadelphia 76ers,115,Washington Wizards,120
6,2017-10-18,Milwaukee Bucks,108,Boston Celtics,100
7,2017-10-18,New Orleans Pelicans,91,Memphis Grizzlies,103
8,2017-10-18,Atlanta Hawks,117,Dallas Mavericks,111
9,2017-10-18,Denver Nuggets,96,Utah Jazz,106


In [103]:
from basketball_reference_scraper.constants import TEAM_ABBR_TO_TEAM, TEAM_TO_TEAM_ABBR
from basketball_reference_scraper.seasons import get_schedule
from basketball_reference_scraper.shot_charts import get_shot_chart
from basketball_reference_scraper.pbp import get_pbp
import pandas as pd
from pprint import pprint
import time
start = time.time()

# Each row is the defensive team
# Each column is the offensive team
# Each element is a list of successful makes by the offensive team against the defensive team

TEAMS = ['ATLANTA', 'BOSTON', 'BROOKLYN', 'CHICAGO', 'CHARLOTTE', 
        'CLEVELAND', 'DALLAS', 'DENVER', 'DETROIT', 
         'GOLDEN STATE', 'HOUSTON', 'INDIANA', 'LA CLIPPERS', 
         'LA LAKERS', 'MEMPHIS', 'MIAMI', 'MILWAUKEE', 'MINNESOTA', 
         'NEW ORLEANS', 'NEW YORK', 'OKLAHOMA CITY', 'ORLANDO', 'PHILADELPHIA', 
         'PHOENIX', 'PORTLAND', 'SACRAMENTO', 'SAN ANTONIO', 'TORONTO', 
         'UTAH', 'WASHINGTON']# 'NEW JERSEY NETS', 
         #'NEW ORLEANS HORNETS', 'NEW ORLEANS OKLAHOMA CITY HORNETS', 'CHARLOTTE BOBCATS', 'SEATTLE SUPERSONICS', 'VANCOUVER GRIZZLIES']

# Because of how pandas converts dictionaries into dataframes,
# the "first" key is actually the column, while the "second" key is the row
# team_a is defending team, team_b is offending team
shots_dict = {team_a: {team_b:[] for team_b in TEAMS} for team_a in TEAMS}


s = get_schedule(2019, playoffs=False) # Schedule of all games in the 2018-2019 regular season
for i, series in s.iterrows():
    # Get play by play for the given game
    try:
        pbp = get_pbp(series['DATE'], 
                    TEAM_TO_TEAM_ABBR[series['HOME'].upper()], 
                    TEAM_TO_TEAM_ABBR[series['VISITOR'].upper()])
        teams = series[['HOME', 'VISITOR']] # teams['HOME'], teams['VISITOR']       
            

SyntaxError: unexpected EOF while parsing (<ipython-input-103-fefb7ce5cff2>, line 37)